# A full training

Şimdi Trainer sınıfını kullanmadan son bölümde yaptığımızla aynı sonuçları nasıl elde edeceğimizi göreceğiz. Yine, bölüm 2'de veri işlemeyi yaptığınızı varsayıyoruz. Burada ihtiyacınız olacak her şeyi kapsayan kısa bir özet bulunmaktadır:

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

2024-08-07 06:57:24.912865: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-07 06:57:24.913007: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-07 06:57:25.039595: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

## Prepare for training

Eğitim döngümüzü gerçekten yazmadan önce, birkaç nesne tanımlamamız gerekecek. Bunlardan ilki, yığınlar üzerinde yineleme yapmak için kullanacağımız dataloader'lardır. Ancak bu dataloader'ları tanımlamadan önce, Trainer'ın bizim için otomatik olarak yaptığı bazı şeylerle ilgilenmek için tokenized_datasets'ımıza biraz postprocessing uygulamamız gerekir. Özellikle, şunları yapmamız gerekir:

- Modelin beklemediği değerlere karşılık gelen sütunları kaldırın (cümle1 ve cümle2 sütunları gibi).
- Sütun etiketini labels olarak yeniden adlandırın (çünkü model argümanın labels olarak adlandırılmasını bekler).
- Veri kümelerinin biçimini, listeler yerine PyTorch tensörleri döndürecek şekilde ayarlayın.

Bizim tokenized_datasets metodumuz bu adımların her biri için bir metoda sahiptir:

In [2]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [3]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

Daha sonra sonucun yalnızca modelimizin kabul edeceği sütunlara sahip olup olmadığını kontrol edebiliriz:

Bu işlem tamamlandığına göre, dataloader'larımızı kolayca tanımlayabiliriz:

In [4]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=16, collate_fn=data_collator
)
val_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=16, collate_fn=data_collator
)
test_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=16, collate_fn=data_collator
)

Veri işlemede herhangi bir hata olup olmadığını hızlıca kontrol etmek için bir yığını şu şekilde inceleyebiliriz:

In [5]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([16]),
 'input_ids': torch.Size([16, 81]),
 'token_type_ids': torch.Size([16, 81]),
 'attention_mask': torch.Size([16, 81])}

Eğitim dataloader'ı için shuffle=True ayarladığımız ve yığın içinde maksimum uzunluğa kadar dolgu yaptığımız için gerçek şekillerin muhtemelen sizin için biraz farklı olacağını unutmayın.

Artık veri ön işlemeyi tamamen bitirdiğimize göre (herhangi bir ML uygulayıcısı için tatmin edici ancak zor bir hedef), modele dönelim. Modeli aynen önceki bölümde yaptığımız gibi örneklendiriyoruz:

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels=2
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Eğitim sırasında her şeyin sorunsuz ilerleyeceğinden emin olmak için yığınımızı bu modele aktarıyoruz:

In [7]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.7047, grad_fn=<NllLossBackward0>) torch.Size([16, 2])


Tüm Transformers modelleri, etiketler sağlandığında kaybı döndürür ve ayrıca logitleri de alırız (partimizdeki her girdi için iki tane, yani 8 x 2 boyutunda bir tensör).

Eğitim döngümüzü yazmaya neredeyse hazırız! Sadece iki şey eksik: bir optimize edici ve bir öğrenme oranı zamanlayıcısı. Trainer'ın elle yaptığını kopyalamaya çalıştığımız için aynı varsayılanları kullanacağız. Trainer tarafından kullanılan optimize edici AdamW, Adam ile aynıdır, ancak ağırlık bozulması düzenlemesi için bir bükülme ile (bkz. Ilya Loshchilov ve Frank Hutter tarafından "Decoupled Weight Decay Regularization"):

In [8]:
import torch.optim as optim

optimizer = optim.AdamW(model.parameters(), lr=5e-5)

Son olarak, varsayılan olarak kullanılan öğrenme oranı zamanlayıcısı, maksimum değerden (5e-5) 0'a doğrusal bir düşüştür. Bunu doğru bir şekilde tanımlamak için, çalıştırmak istediğimiz epok sayısı ile eğitim yığınlarının sayısının (eğitim veri yükleyicimizin uzunluğu) çarpımı olan eğitim adımlarının sayısını bilmemiz gerekir. Trainer varsayılan olarak üç epok kullanır, bu yüzden biz de bunu takip edeceğiz:

In [9]:
from transformers import get_scheduler

num_epochs = 10
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

2300


## The training loop

Son bir şey: GPU'ya erişimimiz varsa onu kullanmak isteyeceğiz (CPU'da eğitim birkaç dakika yerine birkaç saat sürebilir). Bunu yapmak için modelimizi ve yığınlarımızı koyacağımız bir cihaz tanımlıyoruz:

In [10]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
device

device(type='cuda')

Artık eğitime hazırız! Eğitimin ne zaman biteceğini anlamak için, tqdm kütüphanesini kullanarak eğitim adımı sayımızın üzerine bir ilerleme çubuğu ekliyoruz:

In [11]:
from tqdm import tqdm

train_losses = []
val_losses = []
best_val_loss = float('inf')
patience = 3

for epoch in range(num_epochs):
    model.train()
    train_running_loss = 0.0
    
    pbar = tqdm(enumerate(train_dataloader), desc=f"Epoch {epoch + 1}/{num_epochs}", unit="batch", total=len(train_dataloader))

    for i, batch in pbar:
        batch = {k: v.to(device) for k, v in batch.items()}
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_running_loss += loss.item()

        pbar.set_postfix(batch_loss=f"{loss.item():.6f}")

    train_epoch_loss = train_running_loss / len(train_dataloader)
    train_losses.append(train_epoch_loss)
    
    # Validation
    model.eval()
    val_running_loss = 0.0
    with torch.no_grad():
        for batch in val_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            val_loss = outputs.loss
            val_running_loss += val_loss.item()

    val_epoch_loss = val_running_loss / len(val_dataloader)
    val_losses.append(val_epoch_loss)
    
    print(f"Epoch {epoch + 1}, Train Loss: {train_epoch_loss:.6f}, Val Loss: {val_epoch_loss:.6f}")
    
    if val_epoch_loss < best_val_loss:
        best_val_loss = val_epoch_loss
        current_patience = 0
    else:
        current_patience += 1

    if current_patience >= patience:
        print(f"Early stopping triggered after {epoch + 1} epochs.")
        break
    

Epoch 1/10: 100%|██████████| 230/230 [00:32<00:00,  7.02batch/s, batch_loss=0.600931]


Epoch 1, Train Loss: 0.521670, Val Loss: 0.397076


Epoch 2/10: 100%|██████████| 230/230 [00:32<00:00,  7.12batch/s, batch_loss=0.269887]


Epoch 2, Train Loss: 0.282621, Val Loss: 0.435394


Epoch 3/10: 100%|██████████| 230/230 [00:32<00:00,  7.12batch/s, batch_loss=0.006450]


Epoch 3, Train Loss: 0.126963, Val Loss: 0.455391


Epoch 4/10: 100%|██████████| 230/230 [00:32<00:00,  7.12batch/s, batch_loss=1.369809]


Epoch 4, Train Loss: 0.070643, Val Loss: 0.478087
Early stopping triggered after 4 epochs.


Eğitim döngüsünün çekirdeğinin giriş bölümündekine çok benzediğini görebilirsiniz. Herhangi bir raporlama istemedik, bu nedenle bu eğitim döngüsü bize modelin nasıl ilerlediği hakkında hiçbir şey söylemeyecek. Bunun için bir değerlendirme döngüsü eklememiz gerekiyor.

## The evaluation loop

Daha önce yaptığımız gibi, Evaluate kütüphanesi tarafından sağlanan bir metrik kullanacağız. **metric.compute()** yöntemini zaten görmüştük, ancak metrikler aslında **add_batch()** yöntemiyle tahmin döngüsünün üzerinden geçerken bizim için yığınları biriktirebilir. Tüm yığınları biriktirdikten sonra, metric.compute() ile nihai sonucu elde edebiliriz. İşte tüm bunların bir değerlendirme döngüsünde nasıl uygulanacağı:

In [12]:
!pip install -q evaluate

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()

for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.8249275362318841, 'f1': 0.8747927031509122}